In [1]:
import json
import pandas as pd
import numpy as np

# Extract

In [3]:
file_dir='/Users/xutian/Desktop/DATA BOOT CAMP/Module 8/Movies-ETL'

with open (f'{file_dir}/wikipedia-movies.json',mode='r') as file:
    wiki_movies_raw=json.load(file) # to load raw JSON into a list of dictionary
    
len(wiki_movies_raw)

7311

In [44]:
# First 5 record
#wiki_movies_raw[:5]

In [45]:
# Last 5 records
#wiki_movies_raw[-5:]

In [46]:
# Some records in the middle
#wiki_movies_raw[3600:3605]

In [47]:
#wiki_movies_raw[0]

In [8]:
kaggle_metadata=pd.read_csv(f'{file_dir}/movies_metadata.csv',low_memory=False)
ratings=pd.read_csv('ratings.csv')

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [13]:
kaggle_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30431,False,NaN,0,"[{'id': 10751, 'name': 'Family'}, {'id': 18, '...",NaN,245017,tt0442001,es,Binta y la gran idea,"Binta, a little girl from Senegal, tells us ab...",...,2004-10-25,0.0,30.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Binta and the Great Idea,False,3.5,3.0
41016,False,"{'id': 259401, 'name': 'The Invisible Man (Uni...",0,"[{'id': 27, 'name': 'Horror'}, {'id': 878, 'na...",NaN,28425,tt0036959,en,The Invisible Man's Revenge,An eccentric scientist helps a fugitive from t...,...,1944-06-09,0.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Invisible Man's Revenge,False,5.4,7.0
14484,False,NaN,70000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://www.sonypictures.com/movies/planet51/,16866,tt0762125,en,Planet 51,When Earth astronaut Capt. Chuck Baker arrives...,...,2009-11-19,104945765.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Something strange is coming to their planet...Us!,Planet 51,False,5.6,573.0
8577,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,10299,tt0058213,en,"Hush... Hush, Sweet Charlotte","An aging, reclusive Southern belle plagued by ...",...,1964-12-15,0.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The years will soon erase a lover's lies...the...,"Hush... Hush, Sweet Charlotte",False,7.0,54.0
36855,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,278778,tt1294141,en,Dancing Ninja,An orphaned boy who dreams of being a ninja ar...,...,2013-10-08,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He's a Kung Fool Fighter!,Dancing Ninja,False,4.7,3.0


# Tranformation/Clean data set

In [14]:
wiki_movies_df=pd.DataFrame(wiki_movies_raw)

In [48]:
#wiki_movies_df.head()

In [49]:
#wiki_movies_df.columns.tolist()

## Structure of list comprehension:
### [expression for element in source_list if some_expression]

In [26]:
wiki_movies=[movie for movie in wiki_movies_raw if ('Director' in movie 
                                                     or 'Directed by' in movie 
                                                     and 'imdb_link' in movie
                                                   and 'No. of episodes' not in movie)]
len(wiki_movies)

7076

In [50]:
wiki_movies_df=pd.DataFrame(wiki_movies)
#wiki_movies_df.head()

In [51]:
#sorted(wiki_movies_df.columns.tolist())

In [38]:
wiki_movies_df[wiki_movies_df['Polish'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
7070,https://en.wikipedia.org/wiki/Cold_War_(2018_f...,2018,https://www.imdb.com/title/tt6543652/,Cold War,Paweł Pawlikowski,"[Tanya Seghatchian, Ewa Puszczyńska]",NaN,NaN,NaN,"[Joanna Kulig, Tomasz Kot, Borys Szyc, Agata K...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zimna wojna


## Delete the columns that with an alternative name, and create a new "alternative name" column in dataframe

In [41]:
def clean_movie(movie):
    movie=dict(movie) # create a non-destructive copy
    alt_titles={}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key]=movie[key]
            movie.pop(key)
    if len(alt_titles)>0:
        movie['alt_titles']=alt_titles
    return movie
# return a single dictionary

In [43]:
# create a new list of dictionary
clean_movies=[clean_movie(movie) for movie in wiki_movies] # Every item in wiki_movies list is a dictionary
wiki_movies_df=pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']